In [45]:
import cv2

dir = './NO_Quad_15/15_2-1/'

image = cv2.imread(dir+'litho_sand_coal_silt.png')

result = image.copy()

# Convert to grayscale and apply Otsu's thresholding
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray,0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

cv2.imwrite(dir+"thresh.png", thresh)

# Detect horizontal lines
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40,1))
detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=1)
detect_horizontal = cv2.dilate(detect_horizontal,horizontal_kernel,iterations = 1)
cnts = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(result, [c], 0, (36,255,12), 1)

cv2.imwrite(dir+"split.png", result)

# # Find maximum of x and minimum
x_start = 0
x_end = 42
y_top = cnts[0][0][0][1]
y_end = image.shape[0]

if y_end - y_top >= 5:
    cv2.imwrite(dir+str(0)+".png", image[y_top:y_end, x_start:x_end])

for i in range(len(cnts)-1):
    y_end = cnts[i][0][0][1]
    y_top= cnts[i+1][0][0][1]
    
    if y_end - y_top >= 5:
        cv2.imwrite(dir+str(i+1)+".png", image[y_top:y_end, x_start:x_end])

y_end = cnts[len(cnts)-1][0][0][1]
y_top = 1

if y_end - y_top >= 5:
    cv2.imwrite(dir+str(len(cnts))+".png", image[y_top:y_end, x_start:x_end])


In [46]:
#prédire la classe de chaque image (mathis) avec model.h5 entrainé avec un ResNet50
#prédiction de plusieurs images d'une même classe
from codecarbon import EmissionsTracker
import os
import numpy as np
import glob
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#class_labels = ['anhydrite', 'calcareous_dolomite', 'chalk', 'chert', 'clay', 'coal_lignite', 'conglomerate', 'dolomite', 'dolomitic_limestone', 'fossiliferous', 'glauconite', 'gypsum', 'limestone', 'marl', 'metamorphic', 'pyrite', 'salt', 'sand', 'shale', 'silt', 'tuff']
#class_labels = ['limestone', 'marl', 'marl_limestone', 'salt', 'sand', 'sand_clay', 'silt']
class_labels = ['coal_lignite', 'marl', 'marl_limestone', 'sand', 'sand_clay']
model = load_model('model_inception.h5')

# Charger toutes les images du dossier test
test_dir = './NO_Quad_15/15_2-1/'

nb_fichiers = len([f for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))])

for i in range(0, nb_fichiers-4): 
    # Ouvrir l'image
    image_files = glob.glob(os.path.join(test_dir, f"{i}.png"))

    for image_file in image_files:
        img = load_img(image_file, target_size=(224, 224))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x)

        # Trouver la classe prédite et afficher le résultat
        class_idx = np.argmax(preds[0])
        predicted_prob = preds[0][class_idx]
        predicted_label = class_labels[class_idx]
            # Vérifier si la probabilité de la classe prédite est inférieure à 50%
        # if predicted_prob < 0.5:
        #     predicted_label = "non identifié"
        # else:
        #     predicted_label = class_labels[class_idx]
        #predicted_label = class_labels[class_idx]
        print("Classe prédite :", predicted_label)

1/1 [==============================] - 1s 1s/step
Classe prédite : sand
1/1 [==============================] - 0s 105ms/step
Classe prédite : coal_lignite
1/1 [==============================] - 0s 94ms/step
Classe prédite : coal_lignite
1/1 [==============================] - 0s 94ms/step
Classe prédite : coal_lignite
1/1 [==============================] - 0s 108ms/step
Classe prédite : coal_lignite
1/1 [==============================] - 0s 103ms/step
Classe prédite : marl
1/1 [==============================] - 0s 104ms/step
Classe prédite : coal_lignite
1/1 [==============================] - 0s 99ms/step
Classe prédite : sand


In [47]:
# Initialiser un dictionnaire qui va contenir le nombre de prédictions pour chaque classe
class_predictions = {m: 0 for m in class_labels}

# Parcourir toutes les images
for i in range(0, nb_fichiers-4):  
    # Ouvrir l'image
    image_files = glob.glob(os.path.join(test_dir, f"{i}.png"))

    for image_file in image_files:
        img = load_img(image_file, target_size=(224, 224))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x)

        # Trouver la classe prédite et incrémenter le compteur correspondant
        class_idx = np.argmax(preds[0])
        predicted_label = class_labels[class_idx]
        class_predictions[predicted_label] += 1

# Trouver la classe prédite qui a été choisie le plus grand nombre de fois
most_predicted_class = max(class_predictions, key=class_predictions.get)
# Trouver le nombre de fois que cette classe a été choisie
most_predictions_count = class_predictions[most_predicted_class]

# Afficher les résultats
print(f"La classe prédite qui a été choisie le plus grand nombre de fois est : {most_predicted_class}")
print(f"Cette classe a été choisie {most_predictions_count} fois.")


1/1 [==============================] - 0s 102ms/step
La classe prédite qui a été choisie le plus grand nombre de fois est : coal_lignite
Cette classe a été choisie 5 fois.


In [48]:
#calcul de la hauteur relative de tous les png
import openpyxl
from PIL import Image
from openpyxl.chart import PieChart, Reference

# Ouvrir les images et stocker les hauteurs relatives pour chaque matériau
heights = {m: [] for m in class_labels}

for i in range(1, nb_fichiers-4): 
    # Ouvrir l'image
    img = Image.open(f"./NO_Quad_15/15_2-1/{i}.png")

    # Calculer la hauteur (en pixels)
    # pas besoin de la largeur qui sera identique
    height = img.height
    valeur_relative = height / 100

    # Ouvrir l'image
    image_files = glob.glob(os.path.join(test_dir, f"{i}.png"))

    for image_file in image_files:
        img = load_img(image_file, target_size=(224, 224))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x)

        # Trouver la classe prédite et afficher le résultat
        class_idx = np.argmax(preds[0])
        predicted_label = class_labels[class_idx]

        # Ajouter la hauteur relative à la liste correspondant au matériau aléatoire choisi
        class_label = predicted_label
        #print(class_label)
        heights[class_label].append(valeur_relative)
    

1/1 [==============================] - 0s 109ms/step


In [49]:
#création du tableur excel et remplissage
#attention aux indices de remplissage de l'excel qui doivent être adaptatifs
# Créer le tableur Excel
wb = openpyxl.Workbook()
ws = wb.active

# Ajouter les hauteurs relatives pour chaque matériau dans le tableur
for i, m in enumerate(class_labels):
    # Ajouter le nom du matériau dans la première colonne
    ws.cell(row=i+2, column=1, value=m)

    # Ajouter les hauteurs relatives dans les colonnes suivantes
    for j, h in enumerate(heights[m]):
        ws.cell(row=i+2, column=j+2, value=h)

# Ajouter la ligne des totaux en bas
#for j in range(2, 7):
#10 etant le nb de fois qu'un matériau est prédit
for j in range(2, most_predictions_count+2):
    sub_sum = sum(ws.cell(row=i, column=j).value or 0 for i in range(2, len(class_labels)+2))
    ws.cell(row=len(class_labels)+2, column=j, value=sub_sum)

# Ajouter la colonne des totaux
for i in range(2, len(class_labels)+3):
    sub_sum = sum(ws.cell(row=i, column=j).value or 0 for j in range(2,  len(class_labels)))
    ws.cell(row=i, column=most_predictions_count+2, value=sub_sum)

# Ajouter la colonne des pourcentages
for i in range(2, len(class_labels)+2):
    total = ws.cell(row=len(class_labels)+2, column=most_predictions_count+2).value
    percent = sum(ws.cell(row=i, column=j).value or 0 for j in range(2, most_predictions_count+2)) / ws.cell(row=len(class_labels)+2, column=most_predictions_count+2).value
    ws.cell(row=i, column=most_predictions_count+3, value=percent).number_format = '0.00%'


# Enregistrer le tableur Excel
#wb.save("resultats_tmp.xlsx")

In [50]:
#génération du graphique camembert
# Créer le graphique camembert
chart = PieChart()
chart.title = "Pourcentage Lithologie"


# Ajouter les données du graphique
labels = Reference(ws, min_col=1, min_row=2, max_row=len(class_labels)+1)
data = Reference(ws, min_col=most_predictions_count+3, min_row=1, max_row=len(class_labels)+1)
chart.add_data(data, titles_from_data=True)
chart.set_categories(labels)

# Ajouter le graphique au tableur
ws.add_chart(chart, "J2")

# Enregistrer le tableur Excel
wb.save("resultats.xlsx")